In [1]:
from core import SimulacionMontecarlo, BlackjackNN, BlackjackCNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

2023-12-28 21:56:41.775913: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-28 21:56:41.824558: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-28 21:56:41.825380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 21:56:42.665707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Ejemplo de uso
simulacion = SimulacionMontecarlo(n_barajas=1, n_simulaciones=1_000_000)
simulacion.simular()

In [ ]:
# s = sum([r[-1] for r in simulacion.resultados])
# s

In [ ]:
def cargar_datos(csv_file):
    df                 = pd.read_csv(csv_file)
    df['Mano Jugador'] = df['Mano Jugador'].apply(json.loads)
    df['Mano Crupier'] = df['Mano Crupier'].apply(json.loads)
    df['Accion']       = df['Accion'].apply(json.loads)
    return df

In [ ]:
df = cargar_datos('resultados_blackjack.csv')
df

In [ ]:
# Crear df_training concatenando las columnas relevantes
df_training = pd.DataFrame(df.apply(lambda x: x['Mano Jugador'] + x['Mano Crupier'] + x['Accion'], axis=1), columns=['Features'])
df_training

In [ ]:
# Crear df_labels con solo la columna Resultado
df_labels = df[['Resultado']]
df_labels

In [ ]:
# Convertir los datos a numpy arrays
X = np.array(df_training['Features'].tolist())
y = np.array(df_labels['Resultado'])

# Convertir las etiquetas a categorías (one-hot encoding)
y = to_categorical(y, num_classes=3)  # Asumiendo que hay tres clases: -1, 0, 1

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y compilar la red neuronal
input_size = X_train.shape[1]  # Número de características
output_size = y.shape[1]       # Número de clases de salida

network = BlackjackNN(input_size, output_size)
network.compile()

In [ ]:
class MetricsCallback(Callback):
    def on_train_begin(self, logs={}):
        self.metrics = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

    def on_epoch_end(self, epoch, logs={}):
        self.metrics['loss'].append(logs.get('loss'))
        self.metrics['accuracy'].append(logs.get('accuracy'))
        self.metrics['val_loss'].append(logs.get('val_loss'))
        self.metrics['val_accuracy'].append(logs.get('val_accuracy'))

metrics_callback = MetricsCallback()

# Entrenar el modelo
history = network.model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[metrics_callback])

# Visualizar las métricas
plt.figure(figsize=(12, 5))

# Pérdida
plt.subplot(1, 2, 1)
plt.plot(metrics_callback.metrics['loss'], label='Entrenamiento')
plt.plot(metrics_callback.metrics['val_loss'], label='Validación')
plt.title('Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# Precisión
plt.subplot(1, 2, 2)
plt.plot(metrics_callback.metrics['accuracy'], label='Entrenamiento')
plt.plot(metrics_callback.metrics['val_accuracy'], label='Validación')
plt.title('Precisión durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()